# open imagej and import libraries

In [ ]:
from cellpose import models, io
from cellpose.io import *
from collections import defaultdict
import geopandas
import glob
import imagej
from jpype import JArray, JInt
import matplotlib.pyplot as plt
import multiprocessing as mp
import numpy as np
import os
import re
import pandas as pd
from pandas import DataFrame
from pathlib import Path
import scyjava
import seaborn
import shutil
import tkinter as tk
from tkinter import filedialog
from PIL import Image
import sys
import os

scyjava.config.add_option('-Xmx60g')
start_dir = os.getcwd()
#ij = imagej.init('sc.fiji:fiji', mode='interactive')
ij = imagej.init('/Users/volkenlab/Documents/Jacques_Data/Fiji.app/', mode='interactive')
## Something about this init() function changes the current working directory.
#ij.getApp().getInfo(True)
ij.ui().showUI()
os.chdir(start_dir)
ij.getVersion() #This is to make sure ImageJ/Fiji opened properly. In case of error, it could display '2.9.0/inactive' instead of the full version

# run trackmate

In [ ]:
Model =  scyjava.jimport('fiji.plugin.trackmate.Model')
Settings= scyjava.jimport('fiji.plugin.trackmate.Settings')
TrackMate = scyjava.jimport('fiji.plugin.trackmate.TrackMate')
Logger= scyjava.jimport('fiji.plugin.trackmate.Logger')
DetectorKeys= scyjava.jimport('fiji.plugin.trackmate.detection.DetectorKeys') 
ExportTracksToXML= scyjava.jimport('fiji.plugin.trackmate.action.ExportTracksToXML') 
TmXmlWriter= scyjava.jimport('fiji.plugin.trackmate.io.TmXmlWriter')
LogRecorder = scyjava.jimport('fiji.plugin.trackmate.util.LogRecorder')
SparseLAPTrackerFactory= scyjava.jimport('fiji.plugin.trackmate.tracking.jaqaman.SparseLAPTrackerFactory')
TMUtils = scyjava.jimport('fiji.plugin.trackmate.util.TMUtils')
HyperStackDisplayer = scyjava.jimport('fiji.plugin.trackmate.visualization.hyperstack.HyperStackDisplayer')
SelectionModel = scyjava.jimport('fiji.plugin.trackmate.SelectionModel')
CellposeDetectorFactory = scyjava.jimport('fiji.plugin.trackmate.cellpose.CellposeDetectorFactory')
FeatureFilter = scyjava.jimport('fiji.plugin.trackmate.features.FeatureFilter')
DisplaySetting = scyjava.jimport('fiji.plugin.trackmate.gui.displaysettings.DisplaySettings')
DisplaySettingsIO = scyjava.jimport('fiji.plugin.trackmate.gui.displaysettings.DisplaySettingsIO')
CaptureOverlayAction = scyjava.jimport('fiji.plugin.trackmate.action.CaptureOverlayAction')
PretrainedModel= scyjava.jimport('fiji.plugin.trackmate.cellpose.CellposeSettings.PretrainedModel')
ThresholdDetectorFactory= scyjava.jimport('fiji.plugin.trackmate.detection.ThresholdDetectorFactory')
TrackScheme = scyjava.jimport('fiji.plugin.trackmate.visualization.trackscheme.TrackScheme')
TrackTableView = scyjava.jimport('fiji.plugin.trackmate.visualization.table.TrackTableView')
AllSpotsTableView = scyjava.jimport('fiji.plugin.trackmate.visualization.table.AllSpotsTableView')
PretrainedModel = scyjava.jimport('fiji.plugin.trackmate.cellpose.CellposeSettings.PretrainedModel')

image = "C:/Users/volkenlab/Documents/Anushka/MAX_Experiment-1647-Split Scenes-02.tif"
out = "C:/Users/volkenlab/Documents/Anushka/"

# Parameters for Detection
# Here, the user can specify parameters for the detection step in Trackmate (Threshold Detector)
dsettings = {
    'TARGET_CHANNEL' : ij.py.to_java(0),
    'SIMPLIFY_CONTOURS' : True,
    'CELLPOSE_MODEL' : PretrainedModel.CUSTOM,
    'CELLPOSE_MODEL_FILEPATH' : "C:/Users/volkenlab/Documents/Anushka/CP_20220523_104016",
    'USE_GPU' : True,
    'OPTIONAL_CHANNEL_2' : ij.py.to_java(0),
    'CELL_DIAMETER' : 10.0,
    'CELLPOSE_PYTHON_FILEPATH' : "C:/Users/volkenlab/anaconda3/envs/cellpose/python.exe",
}

# Parameters for Tracking
# Here, the user can specify parameters for the tracking step in Trackmate (LAP Tracker)
frame_gap = 2
tsettings = {
    'LINKING_MAX_DISTANCE' : 10.0,
    'ALLOW_GAP_CLOSING' : True,
    'GAP_CLOSING_MAX_DISTANCE' : 15.0,
    'MAX_FRAME_GAP' : ij.py.to_java(2),
    'ALLOW_TRACK_SPLITTING' : False,
    'SPLITTING_MAX_DISTANCE' : 15.0,
    'ALLOW_TRACK_MERGING' : False,
    'ALTERNATIVE_LINKING_COST_FACTOR' : 1.05,
    'MERGING_MAX_DISTANCE' : 15.0,
    'CUTOFF_PERCENTILE' : 0.9,
}

if (image[len(image)-4:] == ".tif"):

    # Open Image
    imp = ij.IJ.openImage(image)
    imp.show()
        
    # Create Model
    model = Model()
    settings = Settings(imp)
        
    # Detector
    settings.detectorFactory = CellposeDetectorFactory()
    for parameter, value in dsettings.items():
        settings.detectorSettings[parameter] = value
        
    # Tracker
    settings.trackerFactory = SparseLAPTrackerFactory()
    settings.trackerSettings = settings.trackerFactory.getDefaultSettings()
    for parameter, value in tsettings.items():
        settings.trackerSettings[parameter] = value
        
    # Execute Tracking
    trackmate = TrackMate(model, settings)
    ok = trackmate.checkInput()
    if not ok:
        sys.exit(str(trackmate.getErrorMessage()))
    ok = trackmate.process()
    if not ok:
        sys.exit(str(trackmate.getErrorMessage()))
    selectionModel = SelectionModel(model)
        
    # Display
    ds = DisplaySettingsIO.readUserDefault()
    displayer = HyperStackDisplayer(model, selectionModel, imp, ds)
    displayer.render()
    displayer.refresh()
    trackscheme = TrackScheme(model, selectionModel, ds)
    trackscheme.render()
        
    # Save Data
    outFile = Path(out+"exportModel.xml")
    writer = TmXmlWriter(outFile)
    writer.appendModel(model)
    writer.appendSettings(settings)
    writer.writeToFile()
    csvFileTracks = Path(out+"exportTracks.csv")
    trackTableView = TrackTableView(model, selectionModel, ds)
    trackTableView.getSpotTable().exportToCsv(csvFileTracks)
    

# sort csv file from trackmate output

In [ ]:
import pandas as pd
df = pd.read_csv("C:/Users/volkenlab/Documents/Anushka/exportTracks.csv")
df.drop([0, 1, 2], axis=0, inplace=True)
df = df.reset_index(drop=True)

for index, row in df.iterrows():
    df.at[index, 'TRACK_ID'] = int(row['TRACK_ID'])
    df.at[index, 'FRAME'] = int(row['FRAME'])

df.sort_values(by=['TRACK_ID', 'FRAME'], inplace=True, ignore_index=True)
df.to_csv(r"C:/Users/volkenlab/Documents/Anushka/exportTracks_edited.csv", index=False)

# obtain ROI set

In [ ]:
IJRoiExporter = scyjava.jimport('fiji.plugin.trackmate.action.IJRoiExporter')
model.setLogger(Logger.IJ_LOGGER)
logger = Logger.IJ_LOGGER
exporter = IJRoiExporter(imp, logger)
exporter.export(model.getSpots().iterable(True))
rm = ij.RoiManager.getRoiManager()
rm.save("C:/Users/volkenlab/Documents/Anushka/RoiSet cells.zip");
rm.runCommand(imp,"Delete");

In [ ]:
from zipfile import ZipFile
with ZipFile("C:/Users/volkenlab/Documents/Anushka/RoiSet cells.zip", 'r') as zip: 
    zip.extractall(path="C:/Users/volkenlab/Documents/Anushka/Roiset cells")

# create csv of tracks by row

In [ ]:
import pandas as pd
data = pd.read_csv("C:/Users/volkenlab/Documents/Anushka/exportTracks_edited.csv")
print(data)

import csv
file = open('da_Tracks.csv', 'w', newline='')
csv_writer = csv.writer(file)

current_track = 0
first = True
string = ''
for index, row in data.iterrows():
    if int(row['TRACK_ID']) == current_track:
        if first:
            string = f'{index+1}'
            first = False
        else:
            string = string + ", " + f'{index+1}'
    else:
        csv_writer.writerow([string])
        print(string)
        print(current_track)

        string = f'{index+1}'
        current_track += 1

file.close()

# add ROIs to ROI Manager

In [50]:
df = pd.read_csv("C:/Users/volkenlab/Documents/Anushka/exportTracks_edited.csv")
roiset = "C:/Users/volkenlab/Documents/Anushka/RoiSet cells/"
for index, row in df.iterrows():
    file = row['LABEL'] + '.roi'
    #imp2 = ij.IJ.openImage(roiset+file)
    rm.open(roiset+file)

# add color overlay for each track

In [51]:
df2 = pd.read_csv("C:/Users/volkenlab/Documents/Anushka/da_Tracks.csv", header=None)

nearest_path = "C:/Users/volkenlab/Documents/Anushka/nearest.csv"
testing = pd.read_csv(nearest_path)
rm = ij.RoiManager.getRoiManager()

import random
colors = ["black", "blue", "cyan", "green", "magenta", "orange", "red", "white", "yellow"]

for index, row in df2.iterrows():
    l = row[0].split(", ")
    print(l)
    random_color = random.choice(colors)
    for cell in l:
        cell_index = int(cell)
        roi = rm.select(cell_index)
        overlay_command = f"Overlay.addSelection('{random_color}', 5);"
        ij.py.run_macro(overlay_command)